### Modified version of notebook 02b_initializing.ipynb; see the cell below titled "What just happened?"

In [1]:
import torch

### Why you need a good init

To understand why initialization is important in a neural net, we'll focus on the basic operation you have there: matrix multiplications. So let's just take a vector `x`, and a matrix `a` initialized randomly, then multiply them 100 times (as if we had 100 layers). 

In [2]:
x = torch.randn(512)
a = torch.randn(512,512)

In [3]:
for i in range(100): x = a @ x

In [4]:
x.mean(),x.std()

(tensor(nan), tensor(nan))

The problem you'll get with that is activation explosion: very soon, your activations will go to nan. We can even ask the loop to break when that first happens:

In [5]:
x = torch.randn(512)
a = torch.randn(512,512)

In [6]:
for i in range(100): 
    x = a @ x
    if x.std() != x.std(): break

In [7]:
i

28

It only takes around 30 multiplications! On the other hand, if you initialize your activations with a scale that is too low, then you'll get another problem:

In [8]:
x = torch.randn(512)
a = torch.randn(512,512) * 0.01

In [9]:
for i in range(100): x = a @ x

In [10]:
x.mean(),x.std()

(tensor(0.), tensor(0.))

Here, every activation vanished to 0. So to avoid that problem, people have come with several strategies to initialize their weight matrices, such as:
- use a standard deviation that will make sure x and Ax have exactly the same scale
- use an orthogonal matrix to initialize the weight (orthogonal matrices have the special property that they preserve the L2 norm, so x and Ax would have the same sum of squares in that case)
- use [spectral normalization](https://arxiv.org/pdf/1802.05957.pdf) on the matrix A  (the spectral norm of A is the least possible number M such that `torch.norm(A@x) <= M*torch.norm(x)` so dividing A by this M insures you don't overflow. You can still vanish with this)

### The magic number for scaling

Here we will focus on the first one, which is the Xavier initialization. It tells us that we should use a scale equal to `1/math.sqrt(n_in)` where `n_in` is the number of inputs of our matrix.

In [11]:
import math

In [12]:
# multiply weights by the scale factor for Xavier initialization
x = torch.randn(512,dtype=torch.float64)
a = torch.randn(512,512,dtype=torch.float64) / math.sqrt(512)

In [13]:
# Now we will simulate a deep network by iteratively applying a linear layer to the input
# note that for n_iter much larger than 100, mean and std start to blow up!
n_layers = 50
for i in range(n_layers): x = a @ x

In [14]:
x.mean(),x.std()

(tensor(0.0704, dtype=torch.float64), tensor(1.6490, dtype=torch.float64))

And indeed it works. That is, it works as long as n_layers doesn't get larger than about 50. Why the mean and std blow up for n_layers larger than 50 is not clear to me at this point. Note that this magic number 1/ math.sqrt(512)  isn't very far from the 0.01 we had earlier.

In [15]:
1/ math.sqrt(512)

0.044194173824159216

But where does it come from? It's not that mysterious if you remember the definition of the matrix multiplication. When we do `y = a @ x`, the coefficients of `y` are defined by

$$y_{i} = a_{i,0} x_{0} + a_{i,1} x_{1} + \cdots + a_{i,n-1} x_{n-1} = \sum_{k=0}^{n-1} a_{i,k} x_{k}$$

or in code:
```
y[i] = sum([c*d for c,d in zip(a[i], x)])
```

Now at the very beginning, our `x` vector has a mean of roughly 0. and a standard deviation of roughly 1. (since we picked it that way).

In [16]:
x = torch.randn(512)
x.mean(), x.std()

(tensor(-0.0674), tensor(1.0235))

NB: This is why it's extremely important to normalize your inputs in Deep Learning, the initialization rules have been designed with inputs that have a mean 0. and a standard deviation of 1.

If you need a refresher from your statistics course, the mean is the sum of all the elements divided by the number of elements (a basic average). The standard deviation shows whether the data points stay close to the mean or are far away from it. It's computed by the following formula:

$$\sigma = \sqrt{\frac{1}{n}\left[(x_{0}-m)^{2} + (x_{1}-m)^{2} + \cdots + (x_{n-1}-m)^{2}\right]}$$

where m is the mean and $\sigma$ (the greek letter sigma) is the standard deviation. To avoid that square root, we also often consider a quantity called the variance, which is $\sigma$ squared. 

Here we have a mean of 0, so the variance is just the mean of x squared, and the standard deviation is its square root.

If we go back to `y = a @ x` and assume that we chose weights for `a` that also have a mean of 0, we can compute the variance of `y` quite easily. Since it's random, and we may fall on bad numbers, we repeat the operation 100 times.

In [17]:
mean,sqr = 0.,0.
n_iter = 1000
for i in range(n_iter):
    x = torch.randn(512,dtype=torch.float64)
    a = torch.randn(512, 512,dtype=torch.float64)
    y = a @ x
    # note .item() extracts a float from a tensor
    mean += y.mean().item()
    sqr  += y.pow(2).mean().item()
mean/n_iter,sqr/n_iter

(-0.0029423062165346912, 512.098355930394)

Now that looks very close to the dimension of our matrix 512. And that's no coincidence! When you compute y, you sum 512 product of one element of a by one element of x. So what's the mean and the standard deviation of such a product of one element of `a` by one element of `x`? We can show mathematically that as long as the elements in `a` and the elements in `x` are independent, the mean is 0 and the std is 1.

This can also be seen experimentally:

In [18]:
mean,sqr = 0.,0.
for i in range(10000):
    x = torch.randn(1)
    a = torch.randn(1)
    y = a*x
    mean += y.item()
    sqr  += y.pow(2).item()
mean/10000,math.sqrt(sqr/10000)

(0.004454867896552917, 1.0046282618339664)

Then we sum 512 of those things that have a mean of zero, and a variance of 1, so we get something that has a mean of 0, and variance of 512. To go to the standard deviation, we have to add a square root, hence `math.sqrt(512)` being our magic number.

If we scale the weights of the matrix `a` and divide them by this `math.sqrt(512)`, it will give us a `y` of scale 1, and repeating the product as many times as we want and it won't overflow or vanish.

### Linear transformation of a scalar input, followed by a ReLU

We can reproduce the previous experiment with a linear transformation followed by a ReLU,. We see that this time, the mean shifts and the variance becomes 0.5. 

In [19]:
mean,sqr = 0.,0.
n_iter = 100000
for i in range(n_iter):
    # intitialize input and weights
    x = torch.randn(1)
    a = torch.randn(1)
    # linear transformation
    y = a*x
    # ReLU
    y = 0 if y < 0 else y.item()
    mean += y
    sqr  += y** 2
mean/n_iter,sqr/n_iter

(0.3210805317212011, 0.5095136192303175)

### What just happened?

If you're like me, you want to understand the mathemagic behind the empirically determined results for $mean$ and $sqr$ in the previous cell. Why did the mean shift, and why did the expectation of y**2 change to 0.5? With a bit of effort, we can find out, and it will be good practice for those who want to develop the ability to read papers.

On the other hand if you'd rather not slog through a a few screen of equations, you can accept the empirical result and move on to the bottom of this cell and read the few lines below the TL;DR...

We start with $a$ and $x$, which are $\textbf{independent Gaussian random variables}$ with zero mean and unit variance, and $y = a \cdot x$

The effect of the Relu layer is that $y$ is drawn from a $\textbf{mixture distribution}$: half the time $x \lt 0$ so $y$ is 'clamped' to (i.e. replaced by) zero; the other half the time $x \ge 0$, so $y$ is drawn from a positive half-Gaussian (ref. https://en.wikipedia.org/wiki/Half-normal_distribution).

How can we calculate $mean = E[y]$, the $\textbf{expectation value of y}$? 

Accounting for the $\textbf{mixture distribution}$, we have

$mean = E[y] = 0.5 \cdot E[0] + 0.5 E[a \cdot x] = 0.5 \cdot 0 + 0.5 E[a] \cdot E[x] = 0.5 E[x]^2$, the last step follows because $a$ and $x$ are independent random variables which have the same distribution. So to calculate $mean$, we need to evaluate $E[x]$, the $\textbf{expectation value}$ of $x$ under the half-Gaussian distribution. 

The $\textbf{normalized probability density function}$ for the half-Gaussian is

$ p_{half}(x) =  \frac{1}{\sigma}\sqrt{\frac{2}{\pi}} \cdot exp{\frac{-x^2}{2\sigma^2}} $, where support is on the positive half-line $0 \leq x < \infty$

By $\textbf{normalized}$, we mean that $\int_0^{\infty} p_{half}(x)dx = \int_0^{\infty}\frac{1}{\sigma}\sqrt{\frac{2}{\pi}} \cdot exp{\frac{-x^2}{2\sigma^2}dx} = 1$

$ E[x] = \frac{1}{\sigma}\sqrt{\frac{2}{\pi}} \cdot \int_0^{\infty} x \cdot exp{\frac{-x^2}{2\sigma^2}dx}   = \sigma \sqrt{\frac{2}{\pi}} = \sqrt{\frac{2}{\pi}}$, since $\sigma = 1$

Therefore $mean = E[y] = 0.5E[x]^2 = \frac{1}{\pi} \approx 0.318$

Next we turn to the calculation of $sqr$, the $\textbf{expectation value}$ of $y^2$. 

$E[y^2] = E[a \cdot x \cdot a \cdot x] = E[a^2] \cdot E[x^2] = E[x^2]^2$; the last step again follows because $a$ and $x$ are independent random variables.

Let's evaluate $E[x^2]$, the expectation value of $x^2$ under the half-Gaussian distribution: 

$ E[x^2] = \frac{1}{\sigma}\sqrt{\frac{2}{\pi}} \cdot \int_0^{\infty} x^2 \cdot exp{\frac{-x^2}{2\sigma^2}dx} $

Reflection of the half-Gaussian distribution about the vertical axis (i.e. extending to the negative half-line) produces a full Gaussian, apart from a scale factor:
$\int_{-\infty}^{\infty} p_{half}(x)dx = 2$, so that $0.5 \cdot \int_{-\infty}^{\infty} p_{half}(x)dx = \int_{-\infty}^{\infty} p_{full}(x)dx = 1$, so 

$ p_{full}(x) =  0.5 \cdot p_{half}(x)$ 

The variance of a zero mean unit normal (full) Gaussian is 1 by definition:

$\int_{-\infty}^{\infty} x^2 \cdot p_{full}(x)dx = 1$

By symmetry we can write this as $2 \cdot \int_{0}^{\infty} x^2 \cdot p_{full}(x)dx = 2\cdot \int_{0}^{\infty} x^2 \cdot 0.5 \cdot p_{half}(x)dx = \int_{0}^{\infty} x^2 \cdot p_{half}(x)dx = 1$

But the last expression is just the expectation value of $x^2$ under the half-Gaussian distribution; so we've shown that $E[x^2] = 1$

Now we can compute $E[y^2]$, from our above work: $E[y^2]= E[x^2]^2 = 1$

Finally, taking account that $E[y^2]$ is to be computed under the $\textbf{mixture distribution}$:

$sqr = 0.5 \cdot 0 + 0.5 E[y^2] = 0. + 0.5 \cdot 1 = 0.5$

TL;DR

We've shown analytically that the output of a linear layer followed by a Relu has $E[mean] = \frac{1}{\pi}$ and $E[sqr] = 0.5$, if the inputs and weights are initialized by sampling from a unit normal Gaussian.

This agrees with the empirical results from the previous cell.

One could easily extend this derivation for the (mathematically simpler) case of initializing with a $\textbf{uniform distribution}$. Try it!


### Linear matrix transformation of a vector input, followed by a ReLU
What happens when we run the experiment on the whole matrix product, instead of on the product of scalars?

$sqr$ is the sum of $512$ independent random variables, each of which has expectation value of 0.5. Since the expectation value of a sum of random variables is the sum of the expectation values , we should get $E[sqr]  = 512 \cdot 0.5 = 256$. Indeed, this turns out to be the case.

However, $mean$ is also the sum of $512$ independent random variables, each of which has expectation of $\frac{1}{\pi}$, as we showed above. Therefore,  $mean$ should scale by $512$ to become $E[mean] =\frac{512}{\pi} \approx 163$. Instead we find that $mean \approx 9$. I cannot explain this; I'm still turning it over in my mind to understand why.

In [20]:
# Since sqr is the sum of 512 random variables, each of value 0.5, I expect it to scale by a factor of 512
#    to become 512*0.5 = 256. This is the case
# I also expected the mean to scale by 512, so that mean = 512/pi ~ 163; I cannot explain why empirically, mean ~ 9
n_iter = 10000
n_dim = 512
mean,sqr = 0.,0.
for i in range(n_iter):
    x = torch.randn(n_dim)
    a = torch.randn(n_dim,n_dim)
    y = a@x
    y = y.clamp(min==0)
    mean += y.mean().item()
    # Note the corrected 
    sqr  += y.pow(2).mean().item()
print(n_dim,mean/n_iter,sqr/n_iter)


512 9.035336665630341 256.6674785644531


### We can scale the weights by a 'magic number' $\sqrt\frac{2}{512}$ chosen to make $E[sqr] = 1$ 


The expectation  value of the product of a scalar $a$ with a random variable $X$ is 

$E(aX) = a \cdot E(X)$


With this rule, we can evaluate the expectation value of $(aX)^2$:

$E[(aX)^2] = E[a^2 \cdot X^2] = a^2 \cdot E[X^2]$. So scaling the weights by a factor of $a$ scales the expectation value by $a^2$

The variance of the product of a scalar $a$ and a random variable $X$ is:

$var(aX)  = a^2 \cdot var(X) $ 



In [21]:
# check: sqr should scale by 2/512 --> 0.5*2/512 = 1
# check: mean should scale by sqrt(2/512) --> 9.016*sqrt(2/512) ~ 0.56
mean,sqr = 0.,0.
n_iter = 1000
n_dim = 512
for i in range(n_iter):
    x = torch.randn(n_dim)
    a = torch.randn(n_dim) * math.sqrt(2/n_dim)
    y = a @ x
    y = y.clamp(min=0)
    mean += y.mean().item()
    sqr  += y.pow(2).mean().item()
mean/n_iter,sqr/n_iter

(0.5986050982549787, 1.0889559798707833)

### Let's see how well the new initialization we designed for a linear layer followed by a ReLU activation works: 
We simulate a deep network by repeatedly applying a linear layer followed by a ReLU activation. After each ReLU, randomly initialize the weights with a standard normal distribution multiplied by our "magic number". 

We find that the mean and variance remain fairly stable for up to about $10$ to  $20$  layers. 

However I'd expected $mean$ and $sqr$ to be stable over **hundreds if not thousands of layers**, and this is clearly not the case. I'm at a loss to explain why.  

In [22]:
# what happens when we repeatedly apply a linear layer followed by a ReLU
# with our new initialization?
# 

# control parameters
n_layers = 10
n_dim = 512
n_iter = 100

# initialize accumulators
yy = 0
zz = 0


# outer loop over n_iter: repeat the experiment n_iter times and take
#     avearages of mean and sqr
for j in range(n_iter):

    # initialize accumulators
    mean, sqr = 0.,0.
    
    # intitialize weights with the "magic number" to impose unit variance
    x = torch.randn(n_dim, dtype=torch.float64)
    a = torch.randn(n_dim,n_dim,dtype=torch.float64) * math.sqrt(2/n_dim)
    
    # simulate the effect of a deep network by repeatedly applying
    #     a linear layer followed by a ReLU
    for i in range(n_layers): 
        x = a @ x
        x = x.clamp(min=0)
        mean = x.mean().item()
        sqr  = x.pow(2).mean().item()

    # accumulate sums of mean and sqr
    yy += mean
    zz += sqr

# averages of mean and sqr over n_iter experiments    
yy/n_iter,zz/n_iter
    
    

(0.5753585098577015, 1.146278557410716)

The math behind is a tiny bit more complex, and you can find everything in the [Kaiming](https://arxiv.org/abs/1502.01852) and the [Xavier](http://proceedings.mlr.press/v9/glorot10a.html) paper but this gives the intuition behind those results.